<a href="https://colab.research.google.com/github/BaidyaSubhadip/turantlo_task/blob/main/Disease_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.head(5)

,Disease_Name,Symptom_List
0,Aarskog syndrome,"['Mildly sunken chest (pectus excavatum) ', ' ..."
1,Aase syndrome,"['Narrow shoulders ', ' Droopy eyelids ', ' Ab..."
2,Abdominal aortic aneurysm,"['Passing out. ', ' Dizziness. ', ' Pain in th..."
3,Abnormal uterine bleeding,"['Tenderness and dryness of the vagina ', ' He..."
4,ABO incompatibility,"['Feeling of ""impending doom"" ', ' Chest pain ..."


In [ ]:
unique_symptoms = []
for symptoms in data['Symptom_List']:
  symptom_list = eval(symptoms)
  unique_symptoms.extend(symptom_list)
unique_symptoms = sorted(set(unique_symptoms))
#print(unique_symptoms)

[' "Floppy" body ', ' "Slapping" gait (walking pattern in which each step makes a slapping noise) ', ' "Sloppy" kneecap -- you can move the kneecap too much from right to left (hypermobile patella) ', ' "Strawberry" tongue, with white coating on the tongue, or visible red bumps on the back of the tongue ', ' 2 to 8 centimeters wide or larger', ' A "popping" sound at the time of injury ', ' A baby while still in the womb (congenital CMV) ', ' A bony bump at the first toe joint, with decreased movement in the toe site.', ' A bump about the size of a pea, but may be as large as a golf ball ', ' A bumpy or cobblestone appearance of the skin', ' A change in alertness, and even coma ', ' A change in eating habits, such as not being hungry or eating more than usual ', ' A chest that sinks in or sticks out, called funnel chest (pectus excavatum) or pigeon breast (pectus carinatum) ', ' A combination of symptoms including itching, nausea, headache, fever, vomiting, muscle spasms, tingling in th

In [ ]:
print(len(unique_symptoms))

9681


In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def tokenize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

In [ ]:
def calculate_similarity(input_tokens, symptom_tokens):
    common_tokens = set(input_tokens) & set(symptom_tokens)
    return len(common_tokens) / (len(set(input_tokens)) + len(set(symptom_tokens)) - len(common_tokens))

In [ ]:
def find_closest_symptoms(input_text, symptoms_list, top_n=5):
    input_tokens = tokenize(input_text)
    similarities = []
    for symptom in symptoms_list:
        symptom_tokens = tokenize(symptom)
        similarity = calculate_similarity(input_tokens, symptom_tokens)
        similarities.append((symptom, similarity))

    # Sort based on similarity score in descending order and get top_n results
    closest_symptoms = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]
    return closest_symptoms


In [ ]:
def process_paragraph(input_text, symptoms_list, top_n=5):
    doc = nlp(input_text)
    all_symptoms = []
    for sent in doc.sents:
        closest_symptoms = find_closest_symptoms(sent.text, symptoms_list, top_n=top_n)
        all_symptoms.extend(closest_symptoms)

    # Remove duplicates and sort symptoms based on the highest similarity score
    unique_symptoms = {symptom: score for symptom, score in all_symptoms}
    sorted_symptoms = sorted(unique_symptoms.items(), key=lambda x: x[1], reverse=True)
    return sorted_symptoms[:top_n]

In [ ]:
input_paragraph = """
I've been noticing some unusual physical characteristics in myself, including a mildly sunken chest and undescended testicles.
Additionally, I've been experiencing mild to moderate cognitive problems.
I've also observed some distinct facial features, such as a belly button that sticks out, a top portion of the ear that's folded over slightly, and a small nose with nostrils tipped forward.
I've also had delayed teeth and a scrotum that surrounds my penis, giving it a "shawl scrotum" appearance.
"""

closest_symptoms = process_paragraph(input_paragraph, unique_symptoms, top_n=5)
print("Closest Symptoms:")
for symptom, similarity in closest_symptoms:
    print(f"{symptom}: {similarity:.4f}")

Closest Symptoms:
 Mild to moderate cognitive problems : 0.5000
 Scrotum surrounds the penis (shawl scrotum) : 0.3636
 Small nose with nostrils tipped forward : 0.2778
Mildly sunken chest (pectus excavatum) : 0.2308
 Mild-to-moderate dementia : 0.2222
